In [1]:
import json
import os
import sys
import pandas as pd
import numpy as np

sys.path.append("..")

files = ['leagues',
 'mapping_data',
 'players',
 'teams',
 'tournaments']


In [40]:
### Helper Functions

def create_handle_map(player_data)->dict:
    player_id_map = {entry['player_id']: entry['handle'] for entry in player_data}
    return player_id_map

def id_to_handle(indices, player_id_map):
    return {index: player_id_map[player_id] for index, player_id in indices.items() if player_id in player_id_map}


Esports Data:

1. leagues - ['name','slug','region','tournaments'] - name and region of the league and then within tournaments is a list of key-value pairs corresponding to {'id':"someid"} `tournaments` has the list of dicts corresponding to tournaments
2. mapping_data - ['esportsGameId','teamMapping','participantMapping'] - Game Id, dictionary containing {"200":"id","100":"id"},participant mapping has a dictionary containing {"3":"id"} for 1-10 as the keys `player_list` has the player ids for a game
3. players - ['player_id','handle','home_team_id'] - first and last name for a player is given too, maybe used to verify the player (esp if they changed handle). `player_map` has the player handles all in one reference
4. teams - ['team_id','name','acronym','slug'] - no nested information, can use this to directly give a name to each team
5. tournaments - ['id','leagueId','slug','startDate','endDate','stages'] - tournament id, league id, name of tournament, start of tournament, end of tournament, stages is a list of dictionaries containing the stages of a tournament (groups, knockout)

5.5 stages - ['slug','sections'] - slug is the name of the stage and sections has a list of dictionaries. sections are the different partitions for the tournament stage (e.g. group A, B, etc)
5.6 sections - ['names','matches','rankings'] - name of the section, list of dictionaries for the matches played and rankings (I think rankings aren't there for the first game since there weren't any games)
5.65 matches - ['id','state','strategy','teams','games']

In [2]:
file = files[4]

with open(f'C:/Users/akmar/PycharmProjects/lolpowerrank/riot_esports_ranking/esports-data/{file}.json', 'r') as f:
    data = json.load(f)



In [41]:
df = pd.read_parquet("C:\\Users\\akmar\\PycharmProjects\\lolpowerrank\\riot_esports_ranking\\combined_data_raw.parquet")

Series([], Name: count, dtype: int64)

In [76]:
data[24]['stages'][1]['sections'][0]['matches'][8]['teams']



IndexError: list index out of range

In [55]:
a =sorted([league['slug'] for league in data if league['leagueId'] ==  '98767991310872058'])
new_id =  '108001239847565215'
set(a)

{'lck_2020_split1',
 'lck_regional_finals_2021',
 'lck_regional_finals_2022',
 'lck_spring_2021',
 'lck_spring_2022',
 'lck_spring_2023',
 'lck_summer_2020',
 'lck_summer_2021',
 'lck_summer_2022',
 'lck_summer_2023'}

In [50]:
b = {league['name']:league["id"] for league in data} 
b

{'LCS': '98767991299243165',
 'LCS Challengers': '109511549831443335',
 'LCS Challengers Qualifiers': '109518549825754242',
 'College Championship': '107898214974993351',
 'CBLOL': '98767991332355509',
 'LCK': '98767991310872058',
 'LCL': '98767991355908944',
 'LCO': '105709090213554609',
 'LEC': '98767991302996019',
 'LJL': '98767991349978712',
 'LLA': '101382741235120470',
 'LPL': '98767991314006698',
 'PCS': '104366947889790212',
 'TCL': '98767991343597634',
 'VCS': '107213827295848783',
 'MSI': '98767991325878492',
 'Worlds': '98767975604431411',
 'All-Star Event': '98767991295297326',
 'EMEA Masters': '100695891328981122',
 'La Ligue Française': '105266103462388553',
 'NLC': '105266098308571975',
 'Elite Series': '107407335299756365',
 'Liga Portuguesa': '105266101075764040',
 'PG Nationals': '105266094998946936',
 'Ultraliga': '105266088231437431',
 'SuperLiga': '105266074488398661',
 'Prime League': '105266091639104326',
 'Hitpoint Masters': '105266106309666619',
 'Esports Balka

In [32]:
df = pd.json_normalize(data,sep="_")
a = pd.json_normalize(df["stages"][0],sep="_")
b = pd.json_normalize(a["sections"][0],sep="_")
c = pd.json_normalize(b["matches"][0],sep="_")
d = pd.json_normalize(c["teams"][0],sep="_")
d2 = pd.json_normalize(c["games"][0],sep="_")
df

,id,leagueId,name,slug,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,Summer 2023,gll_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,Summer 2023,elite_series_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,Summer 2023,superliga_summer_2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,Summer 2023,ultraliga_summer_2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."
...,...,...,...,...,...,...,...,...
243,103462439438682788,98767991299243165,LCS Spring 2020,lcs_spring_2020,lol,2020-01-24,2020-04-20,"[{'name': 'Regular Season', 'type': None, 'slu..."
244,104253013242629769,101097443346691685,TAL Summer 2020,tal_summer_2020,lol,2020-06-08,2020-08-10,"[{'name': 'Regular Season', 'type': None, 'slu..."
245,105386292089445027,98767991295297326,ase_2020,ase_2020,lol,2020-12-18,2020-12-20,"[{'name': 'east', 'type': None, 'slug': 'east'..."
246,105562379823435874,105266118689416013,baltic_masters_2021_spring,baltic_masters_2021_spring,lol,2021-01-16,2021-03-12,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [48]:
player_list = []
for i in range(0,len(data),1000):
    player_list.append(data[i]["participantMapping"])

In [49]:
len(player_list)

26

In [41]:
player_map = create_handle_map(data)

In [73]:
player_map_df = pd.DataFrame.from_dict(player_map,'index')
player_map_df.columns = ["Player"]
player_map_df.to_csv("player_map.csv")

In [59]:
id_to_handle(player_list[2],player_map)

{'3': 'Bjergsen',
 '5': 'Biofrost',
 '10': 'IgNar',
 '2': 'Dardoch',
 '1': 'BrokenBlade',
 '9': 'WildTurtle',
 '7': 'Santorin',
 '8': 'PowerOfEvil',
 '6': 'Solo',
 '4': 'Kobbe'}

In [32]:
player_mappings = {}
for j in range(1,10):
    
for i in range(len(data)):
    if data[i]["player_id"] == 

4222

In [25]:
tournaments = []
for i in range(38):

    tournaments.append(data[i]['name'])

tournaments

['LCS',
 'LCS Challengers',
 'LCS Challengers Qualifiers',
 'College Championship',
 'CBLOL',
 'LCK',
 'LCL',
 'LCO',
 'LEC',
 'LJL',
 'LLA',
 'LPL',
 'PCS',
 'TCL',
 'VCS',
 'MSI',
 'Worlds',
 'All-Star Event',
 'EMEA Masters',
 'La Ligue Française',
 'NLC',
 'Elite Series',
 'Liga Portuguesa',
 'PG Nationals',
 'Ultraliga',
 'SuperLiga',
 'Prime League',
 'Hitpoint Masters',
 'Esports Balkan League',
 'Greek Legends League',
 'Arabian League',
 'LCK Academy',
 'LJL Academy',
 'LCK Challengers',
 'CBLOL Academy',
 'North Regional League',
 'South Regional League',
 'TFT Rising Legends']

In [14]:
def inspect_structure(d, indent=0):
    if isinstance(d, dict):
        for key, value in d.items():
            print('  ' * indent + f'Key: {key}, Type: {type(value)}')
            inspect_structure(value, indent + 1)
    elif isinstance(d, list) and len(d) > 0:
        print('  ' * indent + f'List of {type(d[0])} (length {len(d)})')
        inspect_structure(d[0], indent + 1)
        
inspect_structure(data)

List of <class 'dict'> (length 38)
  Key: id, Type: <class 'str'>
  Key: name, Type: <class 'str'>
  Key: slug, Type: <class 'str'>
  Key: sport, Type: <class 'str'>
  Key: image, Type: <class 'str'>
  Key: lightImage, Type: <class 'str'>
  Key: darkImage, Type: <class 'str'>
  Key: region, Type: <class 'str'>
  Key: priority, Type: <class 'int'>
  Key: displayPriority, Type: <class 'dict'>
    Key: position, Type: <class 'int'>
    Key: status, Type: <class 'str'>
  Key: tournaments, Type: <class 'list'>
    List of <class 'dict'> (length 25)
      Key: id, Type: <class 'str'>


# Data work

In [63]:
import mapping as mp
import logging
sys.path.append("..")

In [77]:
# Load all the data
filepath = f"C:/Users/akmar/PycharmProjects/lolpowerrank/riot_esports_ranking/esports-data/"
map_data = mp.game_mapping(filepath=filepath)
map_data = mp.player_mapping(map_data,filepath)
map_data = mp.team_mapping(map_data,filepath)
league_data = mp.league_mapping(filepath)
tournament_data = mp.tournament_mapping(filepath)
# Merges
tournament_data =  tournament_data.merge(map_data,how="left",left_on='game_id',right_on='esportsGameId')
tournament_data = tournament_data.merge(league_data,how="left",on='tournament_id')
#tournament_data.to_parquet("combined_data_raw.parquet")
# Data Validation
t_d_unique = set(tournament_data['game_id'])
m_d_unique = set(map_data['esportsGameId'])
t_d_diff = t_d_unique.difference(m_d_unique)
m_d_diff = m_d_unique.difference(t_d_unique)
t_d_unique_2 = set(tournament_data['tournament_id'])
l_d_unique_2 = set(league_data['tournament_id'])
t_d_diff_2 = t_d_unique_2.difference(l_d_unique_2)
l_d_diff = l_d_unique_2.difference(t_d_unique_2)

TypeError: 'NoneType' object is not subscriptable